# Keras

[Keras](https://keras.io/zh/#keras-python)是一个用Python编写的高级神经网络API,它能够以TensorFlow,CNTK,或者Theano作为后端运行.

keras分为两种模型:

(1) 简单型:Sequential

(1.1) 初始化 Sequential,增加layer

(1.2) 编译模型,增加loss function,optimizer

(1.3) 训练模型


(2) 常用型:函数式API

(2.1) 定义模型中需要的各种函数和layer

(2.2) 建立模型

(2.3) 编译模型并提供loss,optimizer,或者调解其他需要的参数

(2.4) 训练模型

In [1]:
import h5py
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### 1 加载数据集

使用的是.h5的文件类型

In [2]:
def load_data():
    '''
    create train set and test set
    make sure you have .h5 file in your dataset
    
    Returns:
    -------
        train_set_x_orig: original train set shape is (209, 64, 64, 3) 
        train_set_y_orig: original train label shape is (209,)
        test_set_x_orig: original test set shape is (50, 64, 64, 3)
        test_set_y_orig: original test label shape is (50,)
        classes: cat or non-cat.
        
        
    Note:
    ----
        (209, 64, 64, 3): 209 picture,64 width,64 height,3 channel.
    '''
    train_dataset = h5py.File('../data_set/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('../data_set/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [3]:
train_x_orig, train_y_orig, test_x_orig, test_y_orig, classes = load_data()

由Keras的Dense矩阵相乘定义我们需要将labels转换为维度为(m,1)的形式.

下面会详细说明为什么要转换

In [4]:
train_x = train_x_orig.reshape(train_x_orig.shape[0],-1) / 255 
train_y = train_y_orig.reshape(1,-1).T
test_y  = test_y_orig.reshape(1,-1).T
test_x = test_x_orig.reshape(test_x_orig.shape[0],-1) / 255
print('Train_x\'s shape:{}'.format(train_x.shape))
print('Test_x\'s shape:{}'.format(test_x.shape))
print("Train_y's shape:{}".format(train_y.shape))
print("Test_y's shape:{}".format(test_y.shape))

Train_x's shape:(209, 12288)
Test_x's shape:(50, 12288)
Train_y's shape:(209, 1)
Test_y's shape:(50, 1)


### 2 Sequential Model

Sequential 模型是Keras中最简单的模型.只需要加入Desen全连接层就可以.不涉及任何跳跃层或者卷积,池化层.


**Ps:**

全连接层,卷积或者池化层后面章节会详细说明.

#### 2.1 initialization Sequential

In [5]:
model = Sequential()

#### 2.2 add layers

定义完毕model以后,我们就可以开始加入全连接层了使用

```python 
add```

在Keras中全连接层使用:

[Dense](https://keras.io/zh/layers/core/#dense):
- units:表示当前的layer的神经元节点,由于我们是LR模型,所以我们只有一层layer和一个神经元节点.

- [activation](https://keras.io/zh/activations/): 对于LR我们选择sigmoid激活函数

    - output = activation(dot(input, kernel) + bias)
    
    - input_shape:输入的数组维度,这里指的是特征数.
    
        - 对于本例子来说会形成(*,12288)的数组,并且由于mini_batch的存在,所以我们不需要指定数据大小.就类似于TF中placeholder的定义.
        
        - 输出就会成为一个(*,1)的输出.
        
        - 所以我们上面的标签才需要定义为(m,1)的形式.
        
- kernel_initializer:
    - 可以使用API定义:keras.initializers.RandomNormal
    
    - 可以直接使用函数名字:random_normal
    
    - 初始化方式还有很多,可以在此处查看[initializer](https://keras.io/zh/initializers/)
    
**Ps:**

mini_batch:后面会详细讲解

In [6]:
model.add(Dense(units=1,activation='sigmoid',input_shape=(12288,),kernel_initializer='random_normal'))

Instructions for updating:
Colocations handled automatically by placer.


### 2.3 difine optimizer and loss

当layer处理完毕了之后就可以开始定义优化器和损失函数

(1) [optimizers](https://keras.io/zh/optimizers/)

(2) [loss](https://keras.io/zh/losses/)

需要注意的是:

这里定义metrics=['accuracy']了之后,在训练完毕后调用模型评估才会有正确率,否则只有loss值


In [7]:
sgd = keras.optimizers.SGD(lr=0.01) # lr: learning rate
model.compile(loss=keras.losses.binary_crossentropy,optimizer=sgd,metrics=['accuracy'])

### 2.4 fit

当上面的步骤全部定义完毕之后就可以开始训练模型了

epochs:迭代次数

**Ps:**

batch_size简单来说就是每一次梯度更新多少数据.在之后我们会慢慢说明

In [8]:
model.fit(x=train_x,y=train_y,batch_size=64,epochs=1000)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
209/209 [==============================] - 0s 686us/step - loss: 0.9331 - acc: 0.5742
Epoch 2/1000
209/209 [==============================] - 0s 74us/step - loss: 2.1464 - acc: 0.5167
Epoch 3/1000
209/209 [==============================] - 0s 79us/step - loss: 1.5504 - acc: 0.5694
Epoch 4/1000
209/209 [==============================] - 0s 91us/step - loss: 1.4985 - acc: 0.6077
Epoch 5/1000
209/209 [==============================] - 0s 112us/step - loss: 1.3812 - acc: 0.5407
Epoch 6/1000
209/209 [==============================] - 0s 113us/step - loss: 1.7251 - acc: 0.5502
Epoch 7/1000
209/209 [==============================] - 0s 116us/step - loss: 2.3384 - acc: 0.5646
Epoch 8/1000
209/209 [==============================] - 0s 100us/step - loss: 1.6518 - acc: 0.5550
Epoch 9/1000
209/209 [==============================] - 0s 89us/step - loss: 1.8195 - acc: 0.5407
Epoch 10/1000
209/209 [==============================] - 0s 87us/

209/209 [==============================] - 0s 72us/step - loss: 0.9924 - acc: 0.6507
Epoch 163/1000
209/209 [==============================] - 0s 66us/step - loss: 0.1585 - acc: 0.9665
Epoch 164/1000
209/209 [==============================] - 0s 68us/step - loss: 0.1701 - acc: 0.9522
Epoch 165/1000
209/209 [==============================] - 0s 64us/step - loss: 0.1570 - acc: 0.9617
Epoch 166/1000
209/209 [==============================] - 0s 71us/step - loss: 0.1668 - acc: 0.9330
Epoch 167/1000
209/209 [==============================] - 0s 66us/step - loss: 0.4499 - acc: 0.7847
Epoch 168/1000
209/209 [==============================] - 0s 67us/step - loss: 0.2756 - acc: 0.8947
Epoch 169/1000
209/209 [==============================] - 0s 63us/step - loss: 0.1786 - acc: 0.9330
Epoch 170/1000
209/209 [==============================] - 0s 72us/step - loss: 1.1064 - acc: 0.6124
Epoch 171/1000
209/209 [==============================] - 0s 73us/step - loss: 1.3175 - acc: 0.6459
Epoch 172/1000


209/209 [==============================] - 0s 65us/step - loss: 0.1235 - acc: 0.9617
Epoch 245/1000
209/209 [==============================] - 0s 66us/step - loss: 0.1128 - acc: 0.9856
Epoch 246/1000
209/209 [==============================] - 0s 68us/step - loss: 0.1146 - acc: 0.9809
Epoch 247/1000
209/209 [==============================] - 0s 65us/step - loss: 0.1290 - acc: 0.9665
Epoch 248/1000
209/209 [==============================] - 0s 70us/step - loss: 0.1672 - acc: 0.9330
Epoch 249/1000
209/209 [==============================] - 0s 71us/step - loss: 0.1158 - acc: 0.9809
Epoch 250/1000
209/209 [==============================] - 0s 68us/step - loss: 0.1137 - acc: 0.9809
Epoch 251/1000
209/209 [==============================] - 0s 68us/step - loss: 0.1267 - acc: 0.9713
Epoch 252/1000
209/209 [==============================] - 0s 70us/step - loss: 0.1507 - acc: 0.9617
Epoch 253/1000
209/209 [==============================] - 0s 70us/step - loss: 0.1247 - acc: 0.9713
Epoch 254/1000


Epoch 326/1000
209/209 [==============================] - 0s 89us/step - loss: 0.0961 - acc: 0.9904
Epoch 327/1000
209/209 [==============================] - 0s 98us/step - loss: 0.0993 - acc: 0.9904
Epoch 328/1000
209/209 [==============================] - 0s 71us/step - loss: 0.0975 - acc: 0.9904
Epoch 329/1000
209/209 [==============================] - 0s 93us/step - loss: 0.1047 - acc: 0.9904
Epoch 330/1000
209/209 [==============================] - 0s 84us/step - loss: 0.0890 - acc: 0.9952
Epoch 331/1000
209/209 [==============================] - 0s 84us/step - loss: 0.0907 - acc: 0.9904
Epoch 332/1000
209/209 [==============================] - 0s 93us/step - loss: 0.0873 - acc: 0.9952
Epoch 333/1000
209/209 [==============================] - 0s 95us/step - loss: 0.1136 - acc: 0.9856
Epoch 334/1000
209/209 [==============================] - 0s 87us/step - loss: 0.0877 - acc: 0.9952
Epoch 335/1000
209/209 [==============================] - 0s 79us/step - loss: 0.0905 - acc: 0.9952


209/209 [==============================] - 0s 64us/step - loss: 0.0736 - acc: 0.9952
Epoch 490/1000
209/209 [==============================] - 0s 74us/step - loss: 0.0712 - acc: 0.9952
Epoch 491/1000
209/209 [==============================] - 0s 69us/step - loss: 0.0650 - acc: 1.0000
Epoch 492/1000
209/209 [==============================] - 0s 77us/step - loss: 0.0649 - acc: 1.0000
Epoch 493/1000
209/209 [==============================] - 0s 65us/step - loss: 0.0643 - acc: 1.0000
Epoch 494/1000
209/209 [==============================] - 0s 68us/step - loss: 0.0686 - acc: 1.0000
Epoch 495/1000
209/209 [==============================] - 0s 72us/step - loss: 0.0667 - acc: 0.9952
Epoch 496/1000
209/209 [==============================] - 0s 73us/step - loss: 0.0649 - acc: 1.0000
Epoch 497/1000
209/209 [==============================] - 0s 73us/step - loss: 0.0650 - acc: 1.0000
Epoch 498/1000
209/209 [==============================] - 0s 68us/step - loss: 0.0768 - acc: 0.9952
Epoch 499/1000


209/209 [==============================] - 0s 76us/step - loss: 0.0582 - acc: 1.0000
Epoch 572/1000
209/209 [==============================] - 0s 76us/step - loss: 0.0572 - acc: 1.0000
Epoch 573/1000
209/209 [==============================] - 0s 71us/step - loss: 0.0568 - acc: 1.0000
Epoch 574/1000
209/209 [==============================] - 0s 69us/step - loss: 0.0611 - acc: 1.0000
Epoch 575/1000
209/209 [==============================] - 0s 59us/step - loss: 0.0574 - acc: 0.9952
Epoch 576/1000
209/209 [==============================] - 0s 66us/step - loss: 0.0577 - acc: 1.0000
Epoch 577/1000
209/209 [==============================] - 0s 75us/step - loss: 0.0582 - acc: 1.0000
Epoch 578/1000
209/209 [==============================] - 0s 74us/step - loss: 0.0612 - acc: 0.9952
Epoch 579/1000
209/209 [==============================] - 0s 69us/step - loss: 0.0588 - acc: 0.9952
Epoch 580/1000
209/209 [==============================] - 0s 70us/step - loss: 0.0637 - acc: 1.0000
Epoch 581/1000


209/209 [==============================] - 0s 107us/step - loss: 0.0838 - acc: 0.9904
Epoch 654/1000
209/209 [==============================] - 0s 113us/step - loss: 0.0525 - acc: 1.0000
Epoch 655/1000
209/209 [==============================] - 0s 106us/step - loss: 0.0564 - acc: 0.9952
Epoch 656/1000
209/209 [==============================] - 0s 118us/step - loss: 0.0529 - acc: 1.0000
Epoch 657/1000
209/209 [==============================] - 0s 112us/step - loss: 0.0521 - acc: 1.0000
Epoch 658/1000
209/209 [==============================] - 0s 81us/step - loss: 0.0526 - acc: 1.0000
Epoch 659/1000
209/209 [==============================] - 0s 77us/step - loss: 0.0512 - acc: 1.0000
Epoch 660/1000
209/209 [==============================] - 0s 78us/step - loss: 0.0538 - acc: 1.0000
Epoch 661/1000
209/209 [==============================] - 0s 77us/step - loss: 0.0508 - acc: 1.0000
Epoch 662/1000
209/209 [==============================] - 0s 78us/step - loss: 0.0527 - acc: 1.0000
Epoch 663/

Epoch 816/1000
209/209 [==============================] - 0s 93us/step - loss: 0.0423 - acc: 1.0000
Epoch 817/1000
209/209 [==============================] - 0s 90us/step - loss: 0.0428 - acc: 1.0000
Epoch 818/1000
209/209 [==============================] - 0s 89us/step - loss: 0.0432 - acc: 1.0000
Epoch 819/1000
209/209 [==============================] - 0s 94us/step - loss: 0.0423 - acc: 1.0000
Epoch 820/1000
209/209 [==============================] - 0s 92us/step - loss: 0.0440 - acc: 1.0000
Epoch 821/1000
209/209 [==============================] - 0s 99us/step - loss: 0.0495 - acc: 1.0000
Epoch 822/1000
209/209 [==============================] - 0s 90us/step - loss: 0.0422 - acc: 1.0000
Epoch 823/1000
209/209 [==============================] - 0s 92us/step - loss: 0.0426 - acc: 1.0000
Epoch 824/1000
209/209 [==============================] - 0s 101us/step - loss: 0.0425 - acc: 1.0000
Epoch 825/1000
209/209 [==============================] - 0s 107us/step - loss: 0.0420 - acc: 1.000

209/209 [==============================] - 0s 85us/step - loss: 0.0389 - acc: 1.0000
Epoch 980/1000
209/209 [==============================] - 0s 76us/step - loss: 0.0363 - acc: 1.0000
Epoch 981/1000
209/209 [==============================] - 0s 79us/step - loss: 0.0378 - acc: 1.0000
Epoch 982/1000
209/209 [==============================] - 0s 86us/step - loss: 0.0364 - acc: 1.0000
Epoch 983/1000
209/209 [==============================] - 0s 85us/step - loss: 0.0376 - acc: 1.0000
Epoch 984/1000
209/209 [==============================] - 0s 87us/step - loss: 0.0449 - acc: 1.0000
Epoch 985/1000
209/209 [==============================] - 0s 105us/step - loss: 0.0363 - acc: 1.0000
Epoch 986/1000
209/209 [==============================] - 0s 96us/step - loss: 0.0360 - acc: 1.0000
Epoch 987/1000
209/209 [==============================] - 0s 87us/step - loss: 0.0363 - acc: 1.0000
Epoch 988/1000
209/209 [==============================] - 0s 92us/step - loss: 0.0363 - acc: 1.0000
Epoch 989/1000

当模型训练完毕了之后,可以使用[evaluate](https://keras.io/zh/models/model/#evaluate)评估模型.

**Ps:**

如果觉得打印太多,那么可以在fit中指定verbose=0.禁止打印输出

### 2.4 evaluate

由于之前在compile加入metrics=['accuracy'],所以现在返回的是两个值loss,accuracy.可以使用model.metrics_names查看返回的名称.

In [9]:
loss, acc = model.evaluate(test_x, test_y, batch_size=50)
print('The test data accuracy:',acc)

50/50 [==============================] - 0s 669us/step
The test data accuracy: 0.6399999856948853


In [10]:
loss, acc = model.evaluate(train_x, train_y, batch_size=50)
print('The test data accuracy:',acc)

209/209 [==============================] - 0s 67us/step
The test data accuracy: 1.0


如果要绘制loss,acc图,可以使用model.history.history['loss'],model.history.history['acc'].

但是个人建议使用[TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard?hl=zh_cn):

TensorBoard是TF的可视化工具,提供了整个网络模型的结构和其他的功能.由于Keras可以使用TF作为后端,故也可以使用TensorBoard.

在下面使用API的案例中会使用TensorBoard.


### 3 Keras API

In [11]:
import keras
from keras.layers import Input,Dense
from keras.models import Model
from keras.callbacks import TensorBoard

### 3.1 Using Keras API

(1)首先建立输入层

```python
Input(shape=(12288,))```

同样由于batch_size的存在,无需指定样本数量

(2) 初始化参数

```python
keras.initializers.random_normal(seed=1)
```

(3) 构建layers并指定激活函数(activation),将输入层放入

```python
Dense(...)```

(4) 构建模型,指定输入和输出数组

```python
Model(inputs=inputs, outputs=predictions)```

(5) 编译(compile)指定optimizer,loss,如果需要在之后打印测试样本正确率指定 metrics=['accuracy']

```python
model.compile(...)```




In [12]:
inputs = Input(shape=(12288,))
kernel_initializer = keras.initializers.random_normal(seed=1)
predictions = Dense(1,activation='sigmoid',kernel_initializer=kernel_initializer,)(inputs)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### 3.2 fit

在训练的时候,我们可以在回调函数(callbacks)中衔接TensorBoard并指定文件位置(log_dir),当模型训练完毕之后,将会在当前文件夹下生成一个mytensorboard的TensorBoard文件.

In [13]:
model.fit(x=train_x,y=train_y,epochs=1000,callbacks=[TensorBoard(log_dir='mytensorboard')],verbose=0)

### 3.3  start up tensorboard

一旦训练完毕,可以使用Terminal进入该文件夹使用```tensorboard --logdir="mytensorboard2"```.

如果启动失败,在Terminal中输入```which tensorboard```查看tensorboard的路径,并进入bin目录下,在使用```tensorboard --logdir="Absolute path"``.启动.

启动成功之后,会给出一个网址,放入Google chrome即可进入tensorboard交互式环境.

![](../picture/03.png)

### 3.4 evaluate

In [14]:
loss,acc = model.evaluate(x=test_x,y=test_y)
print('The test data accuracy:',acc)

50/50 [==============================] - 0s 907us/step
The test data accuracy: 0.6799999952316285


In [15]:
loss,acc = model.evaluate(x=train_x,y=train_y)
print('The train data accuracy:',acc)

209/209 [==============================] - 0s 45us/step
The train data accuracy: 1.0


In [16]:
model.metrics_names

['loss', 'acc']

# Summary

Keras编写非常简单和友好,对于使用Sequential Model或者API模式,可以根据情况选择,个人建议最好熟悉API模式,对于以后的深层神经网络的搭建会有好处.

关于更多使用Tensorbord将在后续章节中编写